<a href="https://colab.research.google.com/github/defyMiy/Logistic-Regression/blob/main/scads_tokenizer_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !wget https://github.com/wongnai/wongnai-corpus/raw/master/review/review_dataset.zip; unzip review_dataset.zip
# !mkdir wongnai_data; ls

--2022-06-25 06:08:37--  https://github.com/wongnai/wongnai-corpus/raw/master/review/review_dataset.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wongnai/wongnai-corpus/master/review/review_dataset.zip [following]
--2022-06-25 06:08:37--  https://raw.githubusercontent.com/wongnai/wongnai-corpus/master/review/review_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14338706 (14M) [application/zip]
Saving to: ‘review_dataset.zip’

review_dataset.zip  100%[===================>]  13.67M  --.-KB/s    in 0.08s   

2022-06-25 06:08:38 (168 MB/s) - ‘review_dataset.zip’ saved [1433870

In [1]:
!pip install pythainlp
!pip install deepcut
!pip install emoji
!pip install oskut

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 5.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 7.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 8.3 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=6c169c20e5cf12d6f53b287499179371e7b933305c48d2e4e1f9b147745850ab
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44.2 MB 1.7 MB/s 
     |████████████████████████████████| 312 kB 53.8 MB/s 
  Created wheel for pyahocorasick: 

In [5]:
import re, emoji, time, oskut
import pandas as pd
import numpy as np
from pythainlp import word_tokenize
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
oskut.load_model(engine='scads')

In [3]:
def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def process_text(text):
    #pre rules
    res = text.lower().strip()
    res = replace_url(res)
    res = replace_rep(res)
    
    #tokenize
    res = [word for word in oskut.OSKut(res) if word and not re.search(pattern=r"\s+", string=word)]
    
    #post rules
    res = ungroup_emoji(res)
    
    return res

In [6]:
# all_df = pd.read_csv('w_review_train.csv', sep=';', header=None)[:25000]
# all_df.columns = ['texts', 'category']
# all_df['processed'] = all_df.texts.map(lambda x: '|'.join(process_text(x)))
# all_df['wc'] = all_df.processed.map(lambda x: len(x.split('|')))
# all_df['uwc'] = all_df.processed.map(lambda x: len(set(x.split('|'))))
# all_df.to_csv('all_df.csv', index=False)

In [3]:
all_df = pd.read_csv('all_df.csv')
all_df

,texts,category,processed,wc,uwc
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3,ร้านอาหาร|ใหญ่|มาก|เลี้ยว|เข้า|มา|เจอ|ห้องน้ำ|...,82,68
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4,อาหาร|ที่|นี่|เป็น|อาหาร|จีน|แคะ|ที่|หากิน|ยาก...,164,118
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3,ปอเปี๊ยะสด|ทุกวันนี้|รู้สึก|ว่า|หากิน|ยาก|(|ร้...,126,96
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5,รัานคัพเค้ก|ใน|เมือง|ไทย|มี|ไม่|มาก|หลายๆ|คน|อ...,229,139
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5,อร่อย|!|xxrep|เดิน|ผ่าน|ทุก|วัน|ไม่|ยัก|รู้|ว่...,66,53
...,...,...,...,...,...
24995,\n\nหลังออกกำลังกายวันนี้หิวจนตาลาย ร้านก๋วยเต...,3,หลัง|ออกกำลัง|กาย|วันนี้|หิว|จน|ตาลาย|ร้าน|ก๋ว...,208,140
24996,อย่างว่าครับคนเรา ... ถ้าเคยกินของอร่อยกว่า แล...,2,อย่าง|ว่า|ครับ|คน|เรา|.|xxrep|ถ้า|เคย|กิน|ของ|...,148,93
24997,ร้านกว้างขวาง จอดรถได้เยอะ ร้านตกแต่งสวย กลางค...,5,ร้าน|กว้างขวาง|จอดรถ|ได้|เยอะ|ร้าน|ตกแต่ง|สวย|...,83,73
24998,อิ่มแค่ชิมทุกอย่าง ก็อิ่มอาหารมีให้เลือกหลากหล...,4,อิ่ม|แค่|ชิม|ทุก|อย่าง|ก็|อิ่มอาหาร|มี|ให้|เลื...,81,62


In [4]:
seed = int(time.time())
train_df, temp = train_test_split(all_df, test_size=0.2, random_state=seed)
valid_df, test_df = train_test_split(temp, test_size=0.5, random_state=seed)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
train_df, valid_df, test_df

(                                                   texts  category  \
 0      อยากทานอะไรที่เร็วๆเพราะมีเวลไม่ค่อยมากและอยาก...         3   
 1      บาร์ริมถนนซึ่งเป็นที่รู้จักกันดีของชาวต่างชาติ...         5   
 2      McDonald ร้านนี้ตั้งอยู่ในย่านการค้า Kadong ใก...         3   
 3      หลังจากที่ไปทานมาเมื่อวันเสาร์ที่ผ่านมา ถึงเวล...         2   
 4      “Good Food, Great Service & Excellent Coffee” ...         5   
 ...                                                  ...       ...   
 19995  ร้านครัวกรรณิการ์ขายอาหารที่ทำจากไก่ทุกเมนู ไม...         3   
 19996  ผมเป็นคนชอบทานก๋วยเตี๋ยวมากครับ...มื้อเช้าและเ...         2   
 19997  มาโลตัส ลำลูกกาคลองสอง เริ่มกระหายน้ำ เป็นคนชอ...         4   
 19998  หิว ช่วงสายๆ ละ หาร้านอาหารจานเดียวแบบกินเดี่ย...         3   
 19999  เปลี่ยนแนวรีวิวมาบ้างเล็กน้อยคะ มารีวิวร้านเหล...         3   
 
                                                processed   wc  uwc  
 0      อยาก|ทาน|อะไร|ที่|เร็วๆ|เพราะ|มีเวล|ไม่|ค่อย|ม...  147   99  
 1    

In [ ]:
y_train = train_df['category']
y_valid = valid_df['category']

tfidf = TfidfVectorizer(tokenizer=process_text, ngram_range=(1,2), min_df=20, sublinear_tf=True)
tfidf_fit = tfidf.fit(all_df['texts'])
text_train = tfidf_fit.transform(train_df['texts'])
text_valid = tfidf_fit.transform(valid_df['texts'])
text_test = tfidf_fit.transform(test_df['texts'])
print(text_train.shape, text_valid.shape)

In [ ]:
scaler = StandardScaler()
scaler_fit = scaler.fit(all_df[['wc','uwc']].astype(float))
print(scaler_fit.mean_, scaler_fit.var_)

In [ ]:
num_train = scaler_fit.transform(train_df[['wc','uwc']].astype(float))
num_valid = scaler_fit.transform(valid_df[['wc','uwc']].astype(float))
num_test = scaler_fit.transform(test_df[['wc','uwc']].astype(float))
print(num_train.shape, num_valid.shape)

In [ ]:
X_train = np.concatenate([num_train,text_train.toarray()],axis=1)
X_valid = np.concatenate([num_valid,text_valid.toarray()],axis=1)
X_test = np.concatenate([num_test,text_test.toarray()],axis=1)
print(X_train.shape, X_valid.shape)

In [ ]:
model = LogisticRegression(C=2., penalty='l2', solver='liblinear', dual=False, multi_class='ovr')
model.fit(X_train,y_train)
model.score(X_valid,y_valid)